In [23]:
import pandas as pd 
import numpy as np 
#np.random.bit_generator = np.random._bit_generator
import pyodbc
from datetime import datetime, timedelta
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
import nltk
import timeit
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
# gensim
from gensim import corpora, models, similarities, matutils
import unicodedata
f
from sklearn.feature_extraction.text import TfidfVectorizer
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PCC2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PCC2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
server = 't5.database.windows.net'
server='DESKTOP-ASD2RK7\SQLEXPRESS'
database = 'T5' 
username = 'T5' 
password = 'My404Data' 
database = 'Riyadh2'
username ='sab'
password = 'tata2015'
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
df= pd.read_sql("select   * from texts inner join articles on t_aid = a_id where a_words>150 and a_words <501 ",cnxn)
df.to_csv("texts3.csv",encoding='utf-8-sig')

In [ ]:
df2= pd.read_sql("select  * from Articles",cnxn)
df2.to_csv("articles.csv",encoding='utf-8-sig')
df2

In [ ]:
df = pd.read_csv("texts.csv")
df

In [ ]:
df3 = pd.merge(df2, df, how='inner', left_on='A_ID', right_on='T_AID')
df3

In [26]:
df3 = df

In [27]:
df3['Words_Number'] = df3['T_Text'].astype(str).apply(lambda x : len(x.split()))
df3['T_Text'] = df3['T_Text'].apply(lambda x:unicodedata.normalize("NFKD", str(x)).replace('\n', ''))
# Wrong Texts (i.e. very long/short texts)
df3 = df3[((df3['Words_Number'] > 150) & (df3['Words_Number'] < 500))]
df3 = df3.reset_index()
df3 = df3.drop_duplicates(subset ="T_Text")
#df3
Articles = list(df3['T_Text'].tolist())

In [6]:
wordtoken = word_tokenize(' '.join(Articles))

In [7]:
len(wordtoken)

1345836

In [44]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words = stopwords.words('arabic'),token_pattern='\\b\w\w+\\b')
#count_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words = stopwords.words('arabic'),token_pattern='\\b[\u0621-\u064A][\u0621-\u064A]+\\b')
 
X = count_vectorizer.fit_transform(Articles)
#cols = count_vectorizer.get_feature_names_out()
cols = count_vectorizer.get_feature_names()


C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [29]:

#print(cols[100064])
#X.toarray()
# without stop words removal we got 138,561
# using arabic stop words we got 138,108
#print(X)
#print(X[1:1])


In [34]:
X.shape

(139950, 11178591)

In [45]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [46]:
doc_word = count_vectorizer.transform(Articles).transpose()

In [17]:
ab  = pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names())
ab['Total'] = ab.sum(axis=1)
ab = ab.sort_values('Total',ascending=False)
ab.head(60)
#ab2
ab2.shape
#ab2.max()
#ab2.argmax()
#ab[ab2.argmax():ab2.argmax()+1]

MemoryError: Unable to allocate 25.8 GiB for an array with shape (697594, 4973) and data type int64

In [47]:
doc_word.shape

(11178591, 139950)

In [48]:
corpus = matutils.Sparse2Corpus(doc_word)
#models.LdaModel()

In [49]:
lda = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=1,iterations=100)

2021-11-11 01:21:05,626 : INFO : using symmetric alpha at 0.05
2021-11-11 01:21:05,628 : INFO : using symmetric eta at 0.05
2021-11-11 01:21:07,015 : INFO : using serial LDA version on this node
2021-11-11 01:21:25,345 : INFO : running online (single-pass) LDA training, 20 topics, 1 passes over the supplied corpus of 139950 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 100x with a convergence threshold of 0.001000
2021-11-11 01:21:26,482 : INFO : PROGRESS: pass 0, at document #2000/139950
2021-11-11 01:21:51,753 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:22:01,933 : INFO : topic #6 (0.050): 0.007*"الا" + 0.004*"لى" + 0.003*"بن" + 0.003*"كورونا" + 0.003*"الله" + 0.002*"المملكة" + 0.002*"با" + 0.001*"خلال" + 0.001*"الصحة" + 0.001*"والا"
2021-11-11 01:22:02,025 : INFO : topic #18 (0.050): 0.006*"الا" + 0.002*"لى" + 0.001*"ية" + 0.001*"كورونا" + 0.001*"خلال" + 0.001*"المملكة" + 0.

2021-11-11 01:24:37,515 : INFO : topic diff=0.027828, rho=0.408248
2021-11-11 01:24:37,796 : INFO : PROGRESS: pass 0, at document #14000/139950
2021-11-11 01:24:54,742 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:25:04,370 : INFO : topic #10 (0.050): 0.003*"جازان" + 0.002*"منطقة جازان" + 0.002*"بجازان" + 0.002*"مير" + 0.002*"بن" + 0.001*"الا" + 0.001*"العلا" + 0.001*"الحيوانية" + 0.001*"الاحتفال" + 0.001*"بمنطقة جازان"
2021-11-11 01:25:04,460 : INFO : topic #13 (0.050): 0.001*"نظام الجامعات" + 0.001*"الجامعات الجديد" + 0.001*"شة" + 0.001*"وقسم" + 0.001*"مجلس شو" + 0.001*"ديربي" + 0.000*"عاي شة" + 0.000*"روني" + 0.000*"ون الجامعات" + 0.000*"وين"
2021-11-11 01:25:04,553 : INFO : topic #3 (0.050): 0.003*"الدولة عضو" + 0.003*"نهيان" + 0.002*"زايد نهيان" + 0.002*"الا ميرة" + 0.002*"جدوى" + 0.002*"ميرة نورة" + 0.001*"راشد مكتوم" + 0.001*"فريق الهلال" + 0.001*"مون" + 0.001*"غاني"
2021-11-11 01:25:04,670 : INFO : topic #8 (0.050): 0.013*"الا" + 0.

2021-11-11 01:27:47,658 : INFO : topic diff=0.008703, rho=0.288675
2021-11-11 01:27:47,949 : INFO : PROGRESS: pass 0, at document #26000/139950
2021-11-11 01:28:04,165 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:28:14,437 : INFO : topic #13 (0.050): 0.001*"وقسم" + 0.001*"ديربي" + 0.001*"شة" + 0.001*"عاي شة" + 0.001*"كاونتي" + 0.000*"ديربي كاونتي" + 0.000*"الله عنها" + 0.000*"شة رضي" + 0.000*"روني" + 0.000*"نظام الجامعات"
2021-11-11 01:28:14,549 : INFO : topic #6 (0.050): 0.030*"بن" + 0.022*"الله" + 0.015*"عبدالعزيز" + 0.014*"مير" + 0.013*"بن عبدالعزيز" + 0.012*"الحرمين" + 0.012*"الشريفين" + 0.011*"الحرمين الشريفين" + 0.011*"الا" + 0.010*"سلمان"
2021-11-11 01:28:14,651 : INFO : topic #7 (0.050): 0.002*"المرمى" + 0.002*"دوري المحترفين" + 0.002*"الجولة الا" + 0.002*"زيدان" + 0.002*"الترجيح" + 0.001*"الكرواتي" + 0.001*"البلجيكي" + 0.001*"الهولندي" + 0.001*"الريال" + 0.001*"الروماني"
2021-11-11 01:28:14,752 : INFO : topic #15 (0.050): 0.001*"ل

2021-11-11 01:30:33,243 : INFO : topic diff=0.005065, rho=0.235702
2021-11-11 01:30:33,552 : INFO : PROGRESS: pass 0, at document #38000/139950
2021-11-11 01:30:48,947 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:30:59,176 : INFO : topic #15 (0.050): 0.001*"لولو" + 0.001*"بلقب الدوري" + 0.001*"طفلا" + 0.001*"وقاد" + 0.001*"الا وروبيين" + 0.001*"بلقب كا" + 0.001*"رصاد الجوية" + 0.000*"فوكس" + 0.000*"بوركينا فاسو" + 0.000*"بوركينا"
2021-11-11 01:30:59,300 : INFO : topic #10 (0.050): 0.002*"بجازان" + 0.002*"الشهيد" + 0.001*"خلال موسم" + 0.001*"الاحتفال" + 0.001*"الاعتدال" + 0.001*"الثروة الحيوانية" + 0.001*"تعازي" + 0.001*"استشهد" + 0.001*"عرب مدير" + 0.001*"طلقت وزارة"
2021-11-11 01:30:59,419 : INFO : topic #19 (0.050): 0.010*"يران" + 0.008*"قوات" + 0.006*"الحوثي" + 0.005*"سورية" + 0.005*"رهابية" + 0.005*"الجيش" + 0.005*"يراني" + 0.005*"القوات" + 0.004*"الا يراني" + 0.004*"الا رهابية"
2021-11-11 01:30:59,511 : INFO : topic #17 (0.050): 0.032

2021-11-11 01:33:40,789 : INFO : topic diff=0.002947, rho=0.204124
2021-11-11 01:33:41,106 : INFO : PROGRESS: pass 0, at document #50000/139950
2021-11-11 01:33:56,849 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:34:07,049 : INFO : topic #13 (0.050): 0.002*"وقسم" + 0.001*"ديربي" + 0.000*"روني" + 0.000*"مجلس شو" + 0.000*"شة" + 0.000*"كاونتي" + 0.000*"ون الجامعات" + 0.000*"وين" + 0.000*"ديربي كاونتي" + 0.000*"عاي شة"
2021-11-11 01:34:07,162 : INFO : topic #8 (0.050): 0.009*"الا" + 0.008*"لى" + 0.006*"العمل" + 0.005*"الهيي" + 0.005*"العامة" + 0.004*"خلال" + 0.004*"القطاع" + 0.004*"الصحية" + 0.004*"المو" + 0.004*"وزارة"
2021-11-11 01:34:07,278 : INFO : topic #1 (0.050): 0.029*"الا" + 0.010*"بن" + 0.005*"المنطقة" + 0.004*"منطقة" + 0.004*"لى" + 0.004*"يس" + 0.004*"مير" + 0.004*"خلال" + 0.004*"دارة" + 0.004*"الدكتور"
2021-11-11 01:34:07,372 : INFO : topic #15 (0.050): 0.002*"لولو" + 0.001*"طفلا" + 0.001*"وقاد" + 0.001*"هايبرماركت" + 0.000*"فوكس" 

...

2021-11-11 01:42:28,987 : INFO : topic diff=0.002366, rho=0.154303
2021-11-11 01:42:29,305 : INFO : PROGRESS: pass 0, at document #86000/139950
2021-11-11 01:42:44,110 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:42:54,328 : INFO : topic #18 (0.050): 0.001*"يس الجمهورية" + 0.001*"فقية" + 0.001*"الا فقية" + 0.001*"والغبار" + 0.001*"للا تربة" + 0.001*"تربة والغبار" + 0.000*"مدى الرو" + 0.000*"تكليفه" + 0.000*"الرياح" + 0.000*"تربة"
2021-11-11 01:42:54,447 : INFO : topic #1 (0.050): 0.026*"الا" + 0.009*"بن" + 0.005*"المنطقة" + 0.005*"دارة" + 0.005*"خلال" + 0.005*"منطقة" + 0.005*"يس" + 0.004*"لى" + 0.004*"ري" + 0.004*"مدير"
2021-11-11 01:42:54,571 : INFO : topic #11 (0.050): 0.037*"الا" + 0.014*"لى" + 0.004*"خلال" + 0.004*"با" + 0.004*"كثر" + 0.003*"نه" + 0.003*"العام" + 0.003*"الري" + 0.003*"يس" + 0.003*"السعودية"
2021-11-11 01:42:54,667 : INFO : topic #2 (0.050): 0.001*"نيمار" + 0.001*"جيرمان" + 0.001*"سان جيرمان" + 0.001*"الذراع" + 0.001*"ب

2021-11-11 01:45:06,023 : INFO : topic diff=0.002434, rho=0.144338
2021-11-11 01:45:06,323 : INFO : PROGRESS: pass 0, at document #98000/139950
2021-11-11 01:45:20,696 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:45:30,847 : INFO : topic #3 (0.050): 0.006*"التهاني" + 0.006*"اليوم الوطني" + 0.002*"نهيان" + 0.002*"مام محمد" + 0.002*"سعود الا" + 0.002*"بمحافظة الطاي" + 0.002*"زايد نهيان" + 0.002*"ميرة نورة" + 0.001*"المحبة" + 0.001*"ووحدة"
2021-11-11 01:45:30,978 : INFO : topic #4 (0.050): 0.035*"الا" + 0.006*"الفريق" + 0.005*"الاتحاد" + 0.005*"لى" + 0.005*"ول" + 0.004*"النادي" + 0.004*"ندية" + 0.004*"السعودي" + 0.004*"القدم" + 0.004*"مام"
2021-11-11 01:45:31,080 : INFO : topic #10 (0.050): 0.006*"الشهيد" + 0.003*"خلال موسم" + 0.003*"الاحتفال" + 0.002*"استشهد" + 0.002*"بجازان" + 0.002*"تعازي" + 0.001*"الاعتدال" + 0.001*"فخر واعتزاز" + 0.001*"ميدان العز" + 0.001*"واعتزاز"
2021-11-11 01:45:31,197 : INFO : topic #19 (0.050): 0.009*"يران" + 0.008

2021-11-11 01:48:08,424 : INFO : topic #3 (0.050): 0.005*"التهاني" + 0.003*"اليوم الوطني" + 0.002*"مام محمد" + 0.002*"ميرة نورة" + 0.002*"سعود الا" + 0.002*"نهيان" + 0.002*"نورة بنت" + 0.002*"مون" + 0.001*"زايد نهيان" + 0.001*"بمحافظة الطاي"
2021-11-11 01:48:09,050 : INFO : topic diff=0.002441, rho=0.136083
2021-11-11 01:48:09,379 : INFO : PROGRESS: pass 0, at document #110000/139950
2021-11-11 01:48:23,697 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:48:33,937 : INFO : topic #11 (0.050): 0.037*"الا" + 0.014*"لى" + 0.004*"با" + 0.004*"خلال" + 0.004*"كثر" + 0.004*"نه" + 0.003*"العام" + 0.003*"الري" + 0.003*"وقال" + 0.003*"يس"
2021-11-11 01:48:34,049 : INFO : topic #19 (0.050): 0.008*"قوات" + 0.007*"القوات" + 0.007*"اليمن" + 0.006*"داعش" + 0.006*"يران" + 0.006*"رهابية" + 0.006*"التحالف" + 0.005*"سورية" + 0.005*"اليمني" + 0.005*"الحوثي"
2021-11-11 01:48:34,151 : INFO : topic #5 (0.050): 0.002*"يس عضاء" + 0.002*"التوزيع" + 0.001*"الا علاف" + 0

2021-11-11 01:51:08,731 : INFO : topic #0 (0.050): 0.001*"القضاة" + 0.001*"الفضيلة" + 0.001*"صحاب الفضيلة" + 0.001*"المحرج" + 0.000*"ناصر المحرج" + 0.000*"اللغز" + 0.000*"للصفحة" + 0.000*"للصفحة الشاعر" + 0.000*"حله" + 0.000*"الفضيلة القضاة"
2021-11-11 01:51:09,367 : INFO : topic diff=0.002426, rho=0.129099
2021-11-11 01:51:10,311 : INFO : PROGRESS: pass 0, at document #122000/139950
2021-11-11 01:51:24,525 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:51:34,880 : INFO : topic #19 (0.050): 0.008*"القوات" + 0.008*"قوات" + 0.007*"داعش" + 0.006*"رهابية" + 0.006*"اليمن" + 0.006*"سورية" + 0.006*"التحالف" + 0.005*"الجيش" + 0.005*"يران" + 0.004*"الحوثي"
2021-11-11 01:51:34,980 : INFO : topic #5 (0.050): 0.002*"التوزيع" + 0.001*"يس عضاء" + 0.001*"الا علاف" + 0.001*"وسط حضور" + 0.001*"علاف" + 0.001*"يستعد" + 0.001*"الا طفاء" + 0.001*"البرتغالي كريستيانو" + 0.001*"لبوم" + 0.001*"مير الباحة"
2021-11-11 01:51:35,086 : INFO : topic #3 (0.050): 0.005*"ال

2021-11-11 01:53:40,453 : INFO : topic #14 (0.050): 0.002*"البنك السعودي" + 0.001*"بنتيجة صفر" + 0.001*"خطف" + 0.001*"العملاق" + 0.001*"بدل الضاي" + 0.001*"هدفي" + 0.001*"الوقت بدل" + 0.001*"العديد الفرص" + 0.000*"السعودي البريطاني" + 0.000*"سجلها"
2021-11-11 01:53:41,065 : INFO : topic diff=0.003212, rho=0.123091
2021-11-11 01:53:41,416 : INFO : PROGRESS: pass 0, at document #134000/139950
2021-11-11 01:53:55,569 : INFO : merging changes from 2000 documents into a model of 139950 documents
2021-11-11 01:54:05,278 : INFO : topic #17 (0.050): 0.040*"الا" + 0.010*"لى" + 0.008*"المملكة" + 0.008*"والا" + 0.005*"العربية" + 0.005*"السعودية" + 0.004*"با" + 0.003*"ية" + 0.003*"للا" + 0.003*"عمال"
2021-11-11 01:54:05,373 : INFO : topic #7 (0.050): 0.002*"حريق" + 0.002*"الجولة الا" + 0.001*"الكرواتي" + 0.001*"الترجيح" + 0.001*"زيدان" + 0.001*"المرمى" + 0.001*"العيبان" + 0.001*"اسبانيا" + 0.001*"دوري المحترفين" + 0.001*"الدولة عضو"
2021-11-11 01:54:05,497 : INFO : topic #19 (0.050): 0.008*"قوات" 

In [21]:
corpus = matutils.Sparse2Corpus(doc_word)
corpus

In [22]:
lda.print_topics()


2021-11-10 23:12:35,320 : INFO : topic #0 (0.100): 0.002*"في الدقيقة" + 0.002*"الدقيقة" + 0.001*"الشوط" + 0.001*"الهلال" + 0.001*"في" + 0.001*"من" + 0.001*"الا" + 0.001*"لوشيسكو" + 0.001*"المباراة" + 0.000*"الشوط الثاني"
2021-11-10 23:12:35,330 : INFO : topic #1 (0.100): 0.024*"في" + 0.022*"الا" + 0.017*"من" + 0.011*"على" + 0.007*"وا" + 0.006*"لى" + 0.005*"بن" + 0.004*"التي" + 0.004*"لا" + 0.004*"عن"
2021-11-10 23:12:35,345 : INFO : topic #2 (0.100): 0.021*"في" + 0.020*"الا" + 0.019*"من" + 0.010*"على" + 0.006*"لى" + 0.006*"وا" + 0.005*"التي" + 0.004*"خلال" + 0.003*"مع" + 0.003*"المملكة"
2021-11-10 23:12:35,355 : INFO : topic #3 (0.100): 0.000*"وادا" + 0.000*"الا" + 0.000*"المنشطات" + 0.000*"في" + 0.000*"وزراء الصحة" + 0.000*"من" + 0.000*"عراض" + 0.000*"الصمغ" + 0.000*"الفم والا" + 0.000*"لمكافحة المنشطات"
2021-11-10 23:12:35,366 : INFO : topic #4 (0.100): 0.000*"الروهينجا" + 0.000*"ميانمار" + 0.000*"قلية" + 0.000*"الا" + 0.000*"في" + 0.000*"محكمة العدل" + 0.000*"من" + 0.000*"الوطني الي

[(0,
  '0.002*"في الدقيقة" + 0.002*"الدقيقة" + 0.001*"الشوط" + 0.001*"الهلال" + 0.001*"في" + 0.001*"من" + 0.001*"الا" + 0.001*"لوشيسكو" + 0.001*"المباراة" + 0.000*"الشوط الثاني"'),
 (1,
  '0.024*"في" + 0.022*"الا" + 0.017*"من" + 0.011*"على" + 0.007*"وا" + 0.006*"لى" + 0.005*"بن" + 0.004*"التي" + 0.004*"لا" + 0.004*"عن"'),
 (2,
  '0.021*"في" + 0.020*"الا" + 0.019*"من" + 0.010*"على" + 0.006*"لى" + 0.006*"وا" + 0.005*"التي" + 0.004*"خلال" + 0.003*"مع" + 0.003*"المملكة"'),
 (3,
  '0.000*"وادا" + 0.000*"الا" + 0.000*"المنشطات" + 0.000*"في" + 0.000*"وزراء الصحة" + 0.000*"من" + 0.000*"عراض" + 0.000*"الصمغ" + 0.000*"الفم والا" + 0.000*"لمكافحة المنشطات"'),
 (4,
  '0.000*"الروهينجا" + 0.000*"ميانمار" + 0.000*"قلية" + 0.000*"الا" + 0.000*"في" + 0.000*"محكمة العدل" + 0.000*"من" + 0.000*"الوطني اليمني" + 0.000*"العدل الدولية" + 0.000*"المسلمة"'),
 (5,
  '0.001*"ون الجامعات" + 0.000*"صعوبة الحصول" + 0.000*"عن عمل" + 0.000*"على عمل" + 0.000*"ومستفيدة" + 0.000*"نظام الجامعات" + 0.000*"البحث عن" + 0.0

In [ ]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
#del lda_corpus

In [ ]:
df[df['T_Text'].str.contains("ية ")].iloc[4].T_Text

In [ ]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [ ]:
df6 = pd.DataFrame(columns=['ArticleID','TopicID','Perc'])
def func(x):
    #pass
    #print(x['index'])
    global df6
    for x1 in x['D']:
        #print(x1[1])
        #df6.iloc[-1] = [0,x1[0],x1[1]]
        df6 = df6.append(pd.DataFrame(columns=['ArticleID','TopicID','Perc'],data=[[x['index'],x1[0],x1[1]]]))
        #df6.index = df6.index+1

In [ ]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:50]
df5 = pd.DataFrame(columns = ["D"],data=zip(lda_docs))
#a = np.array(lda_docs[2])
#a[:,1].max()

#df5['Perc'] = df5['D'].apply(lambda x:list(zip(x))[0][0][1])
df5 = df5.reset_index()
df5.apply(lambda x:func(x),axis=1)
df5.head()
df6.head()

In [ ]:
#df3 = df3.reset_index()
df3.head()

In [ ]:
#df6[df6['TopicID']==12]
df7 = pd.merge(df6,df3,how='left',left_on='ArticleID',right_on='index')
df7[df7['TopicID']==9][['ArticleID','TopicID','Perc','A_Cat','A_Title1','A_Title2','T_Text']]

#df7[df7['Index']==4][['ArticleID','TopicID','Perc','A_Cat','A_Title1','A_Title2','T_Text']]
Articles[4]

In [ ]:
Articles[100]

In [50]:
x=lda.show_topics(num_topics=12, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

17::['الا', 'لى', 'المملكة', 'والا', 'العربية']
12::['حلب', 'والمقاييس', 'السوري لحقوق', 'للمواصفات والمقاييس', 'كفاءة الطاقة']
11::['الا', 'لى', 'با', 'خلال', 'نه']
5::['التوزيع', 'يس عضاء', 'الا طفاء', 'وسط حضور', 'راكب']
14::['البنك السعودي', 'بنتيجة صفر', 'خطف', 'العملاق', 'بدل الضاي']
0::['المحرج', 'الفضيلة', 'القضاة', 'صحاب الفضيلة', 'ناصر المحرج']
4::['الا', 'الفريق', 'الاتحاد', 'النادي', 'ول']
7::['الجولة الا', 'زيدان', 'حريق', 'المرمى', 'الترجيح']
10::['الشهيد', 'الاحتفال', 'خلال موسم', 'بجازان', 'استشهد']
6::['بن', 'مير', 'الا', 'الله', 'عبدالعزيز']
1::['الا', 'بن', 'خلال', 'دارة', 'يس']
3::['التهاني', 'اليوم الوطني', 'مون', 'نهيان', 'سعود الا']

الا لى المملكة والا العربية
حلب والمقاييس السوري لحقوق للمواصفات والمقاييس كفاءة الطاقة
الا لى با خلال نه
التوزيع يس عضاء الا طفاء وسط حضور راكب
البنك السعودي بنتيجة صفر خطف العملاق بدل الضاي
المحرج الفضيلة القضاة صحاب الفضيلة ناصر المحرج
الا الفريق الاتحاد النادي ول
الجولة الا زيدان حريق المرمى الترجيح
الشهيد الاحتفال خلال موسم بجاز